## Importing the libraries

In [1]:
import numpy as np
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(dataset.shape, X.shape, y.shape)

(10000, 14) (10000, 10) (10000,)


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
label_encoder_X = LabelEncoder()
X[:, 2] = label_encoder_X.fit_transform(X[:, 2])

ct = ColumnTransformer(
    transformers=[("Geo", OneHotEncoder(), [1])],
    remainder="passthrough",
)
X = ct.fit_transform(X)
X = X[:, 1:]

## Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
print(X_train)

[[0.0 1.0 667 ... 1 0 163830.64]
 [1.0 0.0 427 ... 1 1 57098.0]
 [0.0 0.0 535 ... 1 0 185630.76]
 ...
 [0.0 0.0 738 ... 1 0 181429.87]
 [0.0 1.0 590 ... 1 1 148750.16]
 [1.0 0.0 623 ... 1 0 118855.26]]


In [7]:
print(y_train)

[0 0 0 ... 0 0 1]


In [8]:
print(X_test)

[[1.0 0.0 597 ... 1 1 192852.67]
 [0.0 0.0 523 ... 1 0 128702.1]
 [0.0 1.0 706 ... 1 1 75732.25]
 ...
 [0.0 1.0 578 ... 1 0 141533.19]
 [1.0 0.0 650 ... 1 1 11276.48]
 [1.0 0.0 573 ... 1 0 192950.6]]


In [9]:
print(y_test)

[0 1 0 ... 0 0 0]


In [10]:
X_test.shape, y_test.shape

((2000, 11), (2000,))

## Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
print(X_train)

[[-0.5698444   1.74309049  0.16958176 ...  0.64259497 -1.03227043
   1.10643166]
 [ 1.75486502 -0.57369368 -2.30455945 ...  0.64259497  0.9687384
  -0.74866447]
 [-0.5698444  -0.57369368 -1.19119591 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [-0.5698444  -0.57369368  0.9015152  ...  0.64259497 -1.03227043
   1.41231994]
 [-0.5698444   1.74309049 -0.62420521 ...  0.64259497  0.9687384
   0.84432121]
 [ 1.75486502 -0.57369368 -0.28401079 ...  0.64259497 -1.03227043
   0.32472465]]


In [13]:
print(X_test)

[[ 1.75486502 -0.57369368 -0.55204276 ...  0.64259497  0.9687384
   1.61085707]
 [-0.5698444  -0.57369368 -1.31490297 ...  0.64259497 -1.03227043
   0.49587037]
 [-0.5698444   1.74309049  0.57162971 ...  0.64259497  0.9687384
  -0.42478674]
 ...
 [-0.5698444   1.74309049 -0.74791227 ...  0.64259497 -1.03227043
   0.71888467]
 [ 1.75486502 -0.57369368 -0.00566991 ...  0.64259497  0.9687384
  -1.54507805]
 [ 1.75486502 -0.57369368 -0.79945688 ...  0.64259497 -1.03227043
   1.61255917]]


## Import the keras library and pachages

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

2023-08-16 11:43:52.516968: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Define a create function for model

In [15]:
def create_model():
    model = Sequential()
    model.add(InputLayer(input_shape=(11, ), name='input_layer'))
    model.add(Dense(units=6, activation='relu', name='hidden_layer_1'))
    model.add(Dense(units=1, activation='sigmoid', name='output_layer'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

## Using cross validation score

In [16]:
classifier = KerasClassifier(model=create_model, batch_size=100, epochs=20)
accuracies = cross_val_score(
    estimator=classifier,
    X=X_train,
    y=y_train,
    cv=4,
    n_jobs=-1,
)

mean = np.array(accuracies).mean()
std = np.array(accuracies).std()
print(f"model accuracy with mean:{mean:.3f} and std: {std:.3f}")

2023-08-16 11:43:56.410184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 11:43:56.506213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 11:43:56.734645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
60/60 [==============================] - 3s 4ms/step - loss: 0.7397 - accuracy: 0.4663
Epoch 2/20
Epoch 2/20
Epoch 2/20
60/60 [==============================] - 0s 3ms/step - loss: 0.5506 - accuracy: 0.7690
Epoch 3/20
60/60 [==============================] - 0s 3ms/step - loss: 0.7233 - accuracy: 0.5493
Epoch 3/20
60/60 [==============================] - 0s 4ms/step - loss: 0.5459 - accuracy: 0.7902
Epoch 3/20
60/60 [==============================] - 0s 5ms/step - loss: 0.6536 - accuracy: 0.5557
Epoch 3/20
60/60 [==============================] - 0s 3ms/step - loss: 0.5084 - accuracy: 0.7937
Epoch 4/20
60/60 [==============================] - 0s 3ms/step - loss: 0.6349 - accuracy: 0.6397
Epoch 4/20
60/60 [==============================] - 0s 4ms/step - loss: 0.5071 - accuracy: 0.7960
Epoch 4/20
60/60 [==============================] - 0s 4ms/step - loss: 0.6028 - accuracy: 0.6498
Epoch 4/20
60/60 [==============================] - 0s 3ms/step